# Collecting Geo Data for Dallas

In [1]:
#importing the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import re #regular expessions

! pip install beautifulsoup4
! pip install requests
import requests #request online data
from bs4 import BeautifulSoup as bs # library for navigating html files

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         102 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0



geographiclib-1.52   | 35 KB     | ##################################### | 100% 
geopy-2.2.0          | 67 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transa

In [2]:
dallas_df = pd.read_csv('dallas_data.csv')
dallas_df.head()

,Area,Neighborhood
0,Plano & North Dallas,Addison
1,Plano & North Dallas,Argyle
2,Plano & North Dallas,Bartonville
3,Plano & North Dallas,Celeste
4,Plano & North Dallas,Coppell


In [3]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [2]:
VERSION = '20180604'
LIMIT = 30

In [4]:
area_longitude_list = list()
area_latitude_list = list()
for area in dallas_df['Area']:
    try:
        address = '{}, Dallas, TX'.format(area)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        area_longitude_list.append(longitude)
        area_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        area_longitude_list.append(longitude)
        area_latitude_list.append(latitude)
print('areas printed')

areas printed


In [5]:
hood_longitude_list = list()
hood_latitude_list = list()
for neighborhood in dallas_df['Neighborhood']:
    try:
        address = '{}, Dallas, TX'.format(neighborhood)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
print('hoods printed')

hoods printed


In [6]:
dallas_df.insert(loc=2,
    column='area_longitude',
    value=area_longitude_list
    )
dallas_df.insert(loc=3,
    column='area_latitude',
    value=area_latitude_list
    )
dallas_df.insert(loc=4,
    column='hood_longitude',
    value=hood_longitude_list
    )
dallas_df.insert(loc=5,
    column='hood_latitude',
    value=hood_latitude_list
    )
dallas_df.head()

,Area,Neighborhood,area_longitude,area_latitude,hood_longitude,hood_latitude
0,Plano & North Dallas,Addison,-96.6999,32.9542,-96.8301,32.9605
1,Plano & North Dallas,Argyle,-96.6999,32.9542,-96.8077,32.8097
2,Plano & North Dallas,Bartonville,-96.6999,32.9542,No Data Found,No Data Found
3,Plano & North Dallas,Celeste,-96.6999,32.9542,-96.6335,32.8867
4,Plano & North Dallas,Coppell,-96.6999,32.9542,-97.0156,32.9553


In [7]:
#to prove df is storing correct values and not rounding them
for arealat in dallas_df['area_latitude'][0:5]:
    print(arealat)

32.9542425
32.9542425
32.9542425
32.9542425
32.9542425


In [9]:
dallas_df = dallas_df.set_index('Area')

In [10]:
dallas_df.to_csv('dallas_geodata.csv')

In [11]:
dallas_df.tail()

,Area,Neighborhood,area_longitude,area_latitude,hood_longitude,hood_latitude
214,Fort Worth,Eastside,-97.0342,32.8369,-96.7138,32.9747
215,Fort Worth,South East,-97.0342,32.8369,-96.621,32.9454
216,Other neighborhoods close to Dallas,Heath,No Data Found,No Data Found,-96.8352,32.6801
217,Other neighborhoods close to Dallas,West,No Data Found,No Data Found,-96.7311,32.9226
218,Other neighborhoods close to Dallas,Parker,No Data Found,No Data Found,-96.8932,32.8797


In [8]:
#folium.map cannot have string values for latitude and longitude
for lat,lng in zip(hood_latitude_list, hood_longitude_list):
    if type(lat)== float or type(lat)==int:
        latitude=lat
        longitude=lng
        
#map each neighborhood and area
map_dallas = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(dallas_df['hood_latitude'], dallas_df['hood_longitude'], dallas_df['Neighborhood']):
    try:
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_dallas)
    except:
        continue
for lat, lng, area in zip(dallas_df['area_latitude'], dallas_df['area_longitude'],dallas_df['Area']):
    try:
        label = '{}'.format(area)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=1.0,
            parse_html=False).add_to(map_dallas)
    except:
        continue
map_dallas